In [ ]:
#pip install --user m2crypto

In [32]:
from geni.aggregate import FrameworkRegistry
from geni.aggregate.context import Context
from geni.aggregate.user import User

def buildContext ():
    framework = FrameworkRegistry.get("portal")()
    framework.cert = "/home/lngo/.cert/cloudlab.pem"
    framework.key = "/home/lngo/.cert/cloudlab.pem"
      
    user = User()
    user.name = "lngo"
    user.urn = "urn:publicid:IDN+ch.geni.net+user+lngo"
    user.addKey("/home/lngo/.ssh/id_rsa.pub")

    context = Context()
    context.addUser(user)
    context.cf = framework
    context.project = "BDSLab"
    
    return context

context = buildContext()
import geni.aggregate.instageni as IG
import pprint
pprint.pprint(IG.GPO.getversion(context))

Error: [('PEM routines', 'PEM_def_callback', 'problems getting password'), ('PEM routines', 'PEM_do_header', 'bad password read'), ('SSL routines', 'SSL_CTX_use_PrivateKey_file', 'PEM lib')]

In [11]:
!module purge; module add anaconda/4.3.0; python cloudlab-geni-examples/singleXenVM.py

/bin/sh: line 0: unset: `spark-submit': not a valid identifier
/bin/sh: line 0: unset: `spark-sql': not a valid identifier
/bin/sh: line 0: unset: `spark-shell': not a valid identifier
Traceback (most recent call last):
  File "cloudlab-geni-examples/singleXenVM.py", line 16, in <module>
    node = request.XenVM("node")
AttributeError: 'Request' object has no attribute 'XenVM'
Exception TypeError: TypeError("'NoneType' object is not callable",) in <bound method TempfileManager.__del__ of <geni.tempfile.TempfileManager object at 0x2b5035ceba10>> ignored


In [13]:
import geni.portal as portal
import geni.rspec.pg as rspec

request = rspec.Request()
node = request.ad

NameError: name 'reqquest' is not defined

In [3]:
import geni.portal as portal
import geni.rspec.pg as rspec

request = rspec.Request()
node1 = request.XenVM("node1")
iface1 = node1.addInterface("if1")

# Specify the component id and the IPv4 address
iface1.component_id = "eth1"
iface1.addAddress(rspec.IPv4Address("192.168.1.1", "255.255.255.0"))

node2 = request.XenVM("node2")
iface2 = node2.addInterface("if2")

# Specify the component id and the IPv4 address
iface2.component_id = "eth2"
iface2.addAddress(rspec.IPv4Address("192.168.1.2", "255.255.255.0"))

link = request.LAN("lan")

link.addInterface(iface1)
link.addInterface(iface2)

portal.context.printRequestRSpec()

AttributeError: 'Request' object has no attribute 'XenVM'

In [15]:
%%writefile hpccsystems.py
import geni.portal as portal
import geni.rspec.pg as rspec
import geni.rspec.igext as IG

pc = portal.Context()
request = rspec.Request()

pc.defineParameter("workerCount",
                   "Number of HPCC thorslaves (multiple of 4)",
                   portal.ParameterType.INTEGER, 4)

pc.defineParameter("controllerHost", "Name of controller node",
                   portal.ParameterType.STRING, "node0", advanced=True,
                   longDescription="The short name of the controller node.  You shold leave this alone unless you really want the hostname to change.")

params = pc.bindParameters()

tourDescription = "This profile provides a configurable HPCCSystems testbed one master and customizable thorslaves (multiples of four)."

tourInstructions = \
  """
### Basic Instructions
Once your experiment nodes have booted, and this profile's configuration scripts have finished deploying HPCCSystems inside your experiment, you'll be able to visit [the ECLWatch interface](http://{host-%s}:8010) (approx. 5-15 minutes).  
""" % (params.controllerHost)

#
# Setup the Tour info with the above description and instructions.
#  
tour = IG.Tour()
tour.Description(IG.Tour.TEXT,tourDescription)
tour.Instructions(IG.Tour.MARKDOWN,tourInstructions)
request.addTour(tour)


# Create a Request object to start building the RSpec.
#request = portal.context.makeRequestRSpec()
#request 
# Create a link with type LAN
link = request.LAN("lan")

# Generate the nodes
for i in range(params.workerCount + 1):
    node = request.RawPC("node" + str(i))
    node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
    iface = node.addInterface("if" + str(i))
    iface.component_id = "eth1"
    iface.addAddress(rspec.IPv4Address("192.168.1." + str(i + 1), "255.255.255.0"))
    link.addInterface(iface)
    
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo adduser --ingroup admin --disabled-password hpcc"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo adduser hpcc sudo"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get update"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo wget http://cdn.hpccsystems.com/releases/CE-Candidate-5.2.2/bin/platform/hpccsystems-platform-community_5.2.2-1trusty_amd64.deb"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo dpkg -i hpccsystems-platform-community_5.2.2-1trusty_amd64.deb"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get -y -f install"))
    getEnvFile = "sudo wget https://raw.githubusercontent.com/clemsonbds/hpccsystems/master/environments/" + str(params.workerCount) + ".xml -O /etc/HPCCSystems/environment.xml"
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command=getEnvFile))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo wget https://raw.githubusercontent.com/clemsonbds/hpccsystems/master/conf/environment.conf -O /etc/HPCCSystems/environment.conf"))
    if i == 0:
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sleep 60"))
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo service hpcc-init start"))

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec(request)

Overwriting hpccsystems.py


In [16]:
!module load anaconda/4.3.0; python hpccsystems.py

<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:emulab="http://www.protogeni.net/resources/rspec/ext/emulab/1" xmlns:jacks="http://www.protogeni.net/resources/rspec/ext/jacks/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <rspec_tour xmlns="http://www.protogeni.net/resources/rspec/ext/apt-tour/1">
    <description type="text">This profile provides a configurable HPCCSystems testbed one master and customizable thorslaves (multiples of four).</description>
    <instructions type="markdown">
### Basic Instructions
Once your experiment nodes have booted, and this profile's configuration scripts have finished deploying HPCCSystems inside your experiment, you'll be able to visit [the ECLWatch interface](http://{host-node0}:8010) (approx. 5-15 minutes).  
</instructions>
  </rspe